# NLP pour l’analyse de critiques de films

## Contexte du projet

En règle générale, le nombre d'avis sur un film peu être important et par conséquent le temps de lecture de chaque commentaire peut être une tâche lourde. Alors comment déterminer de manière rapide si un film a eu du succès auprès des spectateurs (ou pas) ? Dans ce contexte, l’idée du projet est d’utiliser des algorithmes d'apprentissage automatique pour la tâche d'analyse de sentiment des spectateurs via leur critique.

Tout d’abord, il sera question que récupérer les données directement du site d’Allociné. En d’autres termes, nous allons scraper les pages qui nous intéressent sur ce site à savoir les critiques des personnes pour le film Inception et Sonic 2.

En navigant sur la page des critiques, vous vous apercevrez que seules deux types d’information ici nous intéresse : la note du spectateur ainsi que son avis. Pourquoi la note ? Parce que nous allons entraîner un modèle de type supervisé et plus précisément un classifieur et donc la note va nous aider à récupérer la classe pour étiqueter le commentaire. Pour cela, nous considérerons qu’une note au-dessus de 3 est considérée comme satisfaisante. Sinon, l’avis est négatif. Ici, nous avons donc réduit le problème à une classification binaire.

## Etape 1 : Web Scraping des données d'avis de spectateurs

Le web scrapping à été réalisé dans le script 'web_scraping.py' et les résultats sont sous forme de dataframes exporté grâce à joblib dans le fichier 'dataframes'

## Etape 2 : Préparation des données

Ayant maintenant nos jeux de données, il faut les préparer afin de pouvoir modéliser notre analyse de sentiments. Pour cela nous allons faire appel à plusieurs techniques :

- Des expressions régulières pour retirer les bruits (ponctuation, etc.) des commentaires.
- Du NLP pour tokeniser et réduire le corpus de chaque commentaire (afin par exemple de ne garder que les mots importants via les stopwords)
  - Des sacs de mots afin de « transformer » nos mots en nombres qui pourront alors être exploités dans un algorithme de Machine learning
  
*Les commentaires seront filtrés à leur essentiel.*

### *Import des librairies :*

In [26]:
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split

### *Import des jeux de données :*

In [7]:
df_sonic = joblib.load('dataframes')['sonic2']
df_inception = joblib.load('dataframes')['inception']

df = pd.concat([df_sonic, df_inception])

In [8]:
df.head()

,Commentaires,Notes
0,\nune bonne suite pour les aventures de sonic ...,4.0
1,\nCette suite de sonic est incroyable !Le fan ...,5.0
2,\nTrès bon film ont retrouve ce qui fait l’esp...,4.5
3,"\nSuper film de ouf, le fait que Knuckles soit...",4.0
4,\nMalgré deux nouveaux personnages de l'univer...,3.0


On remarque de suite que les commentaires ne sont pas encore bien mis en forme, il faudra arranger cela par la suite.

In [10]:
df.isna().sum()

Commentaires    0
Notes           0
dtype: int64

Aucune valeur nulle.

In [12]:
df.shape

(7350, 2)

On a un total de 7350 commentaires dans notre dataset

### *Gestion du bruit des commentaires :*

In [23]:
def standardize_text(df, content_field):
    df[content_field] = df[content_field].str.replace(r"http\S+", "")
    df[content_field] = df[content_field].str.replace(r"http", "")
    df[content_field] = df[content_field].str.replace(r"@\S+", "")
    df[content_field] = df[content_field].str.replace(r"[0-9(),!?@\'\.\-\`\"\_\n]", " ")
    df[content_field] = df[content_field].str.replace(r"@", "at")
    df[content_field] = df[content_field].str.lower()
    return df

In [25]:
standardize_text(df, 'Commentaires')
df.head()

/tmp/ipykernel_5550/492986926.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"http\S+", "")
/tmp/ipykernel_5550/492986926.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"@\S+", "")
/tmp/ipykernel_5550/492986926.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"[0-9(),!?@\'\.\-\`\"\_\n]", " ")


,Commentaires,Notes
0,une bonne suite pour les aventures de sonic a...,4.0
1,cette suite de sonic est incroyable le fan q...,5.0
2,très bon film ont retrouve ce qui fait l’espr...,4.5
3,super film de ouf le fait que knuckles soit ...,4.0
4,malgré deux nouveaux personnages de l univers...,3.0


### *Séparation du jeu de données :*

Les prochaines étapes de préparation ne doivent s'appliquer qu'au jeu d'entraînement, il nous faut garder un jeu de test intact.

In [31]:
y = df['Notes']
X = df.drop('Notes', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Notre jeu d'entrainement contient {X_train.shape[0]} commentaires et celui de test en contient {X_test.shape[0]}")

Notre jeu d'entrainement contient 5880 commentaires et celui de test en contient 1470
